In [199]:

from pre_process_csv.PreProcess import PreProcess
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter,find_peaks
from numpy import linalg 
import pandas as pd
import numpy as np
%matplotlib qt


exp_dir = 'H:/My Drive/dark 2022/csv_dark'
exp_name = ['2022_02_03']

pert_list = ['']
smoothing_config = {'smooth_window_body': [211,511,1011], 'smooth_poly_body':[4,2,2], 'smooth_window_wing': [15,15,15],
                'smooth_poly_wing': [2,2,2], 'zscore_window':200,'zscore_threashold': 3,
                'interp_method' : 'cubic'}


pert_for_attr = ''
hdf5_file = f'{exp_name[0]}{pert_list[0]}'
pertubation_name = hdf5_file.split('_')[-1]
experiment = PreProcess(exp_dir,exp_name[0],pertubation_name, smoothing_config, hdf5_file)

mov = '11'
experiment.load_csv(mov)
experiment.filter_body_wing_and_derive()


def prop_sg(prop_name,prop_to_sg):
    
    prop = experiment.filter_and_derive(prop_to_sg,experiment.angles['frames'],experiment.angles['time'],'smooth_window_body',
                                            'smooth_poly_body',derivetive_name = ['','_dot','_dot_dot'],asign_frames_time = True)
    
    fig,ax = plt.subplots(3,1,sharex=True)
    ax[0].plot(experiment.angles['time'],prop_to_sg[prop_name],'*'),ax[0].title.set_text(prop_name)

    ax[0].plot(experiment.angles['time'],prop[f'{prop_name}'])

    ax[1].plot(experiment.angles['time'],prop[f'{prop_name}_dot']),ax[1].title.set_text(f'{prop_name}_dot')
    ax[2].plot(experiment.angles['time'],prop[f'{prop_name}_dot_dot']),ax[2].title.set_text(f'{prop_name}_dot_dot')
    return prop


def project_cm(cm):
    ref_frame = np.where(experiment.angles['time'] == 0)[0]
    vector_to_project = experiment.vectors[['X_x_body','X_y_body','X_z_body']]

    ref_axes = vector_to_project.iloc[ref_frame].to_numpy()
    x_project_on_z = ref_axes * [0,0,1]
    x_axis_on_xy = (ref_axes - x_project_on_z)/linalg.norm(ref_axes-x_project_on_z )[np.newaxis].T # project the new axis to XY plane
    x_axis_on_xy = np.tile(x_axis_on_xy,(len(cm),1))

    return pd.DataFrame(np.sum(x_axis_on_xy * cm,axis = 1)[np.newaxis,:].T,columns= ['projected_cm_dot'])


In [172]:
smooth_x = prop_sg('CM_real_x_body',experiment.angles[['CM_real_x_body']])
smooth_y = prop_sg('CM_real_y_body',experiment.angles[['CM_real_y_body']])
smooth_z = prop_sg('CM_real_z_body',experiment.angles[['CM_real_z_body']])


In [200]:
experiment.angles.to_csv('angles_dark_mov11.csv')

In [193]:


smoothing_config = {'smooth_window_body': [73*7]*3, 'smooth_poly_body':[3,3,3], 'smooth_window_wing': [15,15,15],
                'smooth_poly_wing': [2,2,2], 'zscore_window':200,'zscore_threashold': 3,
                'interp_method' : 'cubic'}
experiment.smoothing_config = smoothing_config


prop_sg('pitch_body',experiment.angles[['pitch_body']])





,pitch_body,pitch_body_dot,pitch_body_dot_dot,frames,time
0,51.893659,175.349006,19913.907462,73,-27.7500
1,51.904657,176.589257,19774.116385,74,-27.6875
2,51.915733,177.820771,19634.325309,75,-27.6250
3,51.926885,179.043548,19494.534232,76,-27.5625
4,51.938113,180.257588,19354.743155,77,-27.5000
...,...,...,...,...,...
6069,57.382851,-123.167327,-12752.010710,6142,351.5625
6070,57.375129,-123.966011,-12805.895536,6143,351.6250
6071,57.367356,-124.768064,-12859.780362,6144,351.6875
6072,57.359532,-125.573484,-12913.665188,6145,351.7500


לשנות את החישוב ממרכז מסה להטלה של מהירות
smoothing_config = {'smooth_window_body': [73*7]*3, 'smooth_poly_body':[3,3,3], 'smooth_window_wing': [15,15,15],
                'smooth_poly_wing': [2,2,2], 'zscore_window':200,'zscore_threashold': 3,
                'interp_method' : 'cubic'}


In [189]:
smoothed  = pd.concat([smooth_x,smooth_y,smooth_z],axis = 1)
proj_new = project_cm(smoothed[['CM_real_x_body_dot','CM_real_y_body_dot','CM_real_z_body_dot']].to_numpy())

proj_old = project_cm(smoothed[['CM_real_x_body','CM_real_y_body','CM_real_z_body']].to_numpy())
proj_old_smooth = prop_sg('projected_cm',proj_old[['projected_cm']])

plt.figure()
plt.plot(smoothed['time'],proj_new)
plt.plot(smoothed['time'],proj_old_smooth['projected_cm_dot'])

In [186]:
proj_old

,projected_cm
0,-0.000301
1,-0.000295
2,-0.000289
3,-0.000283
4,-0.000277
...,...
6069,-0.008355
6070,-0.008357
6071,-0.008360
6072,-0.008362


In [169]:
 

smoothing_config = {'smooth_window_body': [73*7]*3, 'smooth_poly_body':[3,3,3], 'smooth_window_wing': [15,15,15],
                'smooth_poly_wing': [2,2,2], 'zscore_window':200,'zscore_threashold': 3,
                'interp_method' : 'cubic'}
experiment.smoothing_config = smoothing_config


prop = experiment.filter_and_derive(experiment.angles['CM_real_x_body'],experiment.angles['frames'],experiment.angles['time'],'smooth_window_body',
                                        'smooth_poly_body',derivetive_name = ['','_dot','_dot_dot'],asign_frames_time = True)

prop = experiment.filter_and_derive('CM_real_x_body',experiment.angles['frames'],experiment.angles['time'],'smooth_window_body',
                                        'smooth_poly_body',derivetive_name = ['','_dot','_dot_dot'],asign_frames_time = True)

prop = experiment.filter_and_derive('CM_real_x_body',experiment.angles['frames'],experiment.angles['time'],'smooth_window_body',
                                        'smooth_poly_body',derivetive_name = ['','_dot','_dot_dot'],asign_frames_time = True)
    
prop_sg('pitch_body',experiment.angles[['pitch_body']])
smooth = prop_sg('CM_real_x_body',experiment.angles[['CM_real_x_body']])





IndexError: tuple index out of range

In [165]:

proj = project_cm(experiment.body[['CM_real_x_body','CM_real_y_body','CM_real_z_body']].to_numpy())

smoothing_config = {'smooth_window_body': [211,311,611], 'smooth_poly_body':[4,2,2], 'smooth_window_wing': [15,15,15],
                'smooth_poly_wing': [2,2,2], 'zscore_window':200,'zscore_threashold': 3,
                'interp_method' : 'cubic'}
experiment.smoothing_config = smoothing_config

cm_projected = experiment.filter_and_derive(proj,experiment.angles['frames'],experiment.angles['time'],'smooth_window_body',
                                           'smooth_poly_body',derivetive_name = ['','_dot','_dot_dot'],asign_frames_time = True)

prop_sg('projected_cm',proj)


KeyError: "['CM_real_y_body_dot', 'CM_real_z_body_dot'] not in index"